In [1]:
import os
import torch
import torchvision
from torchvision import datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import requests
from zipfile import ZipFile
from io import BytesIO
import numpy as np
import zipfile
import os


zip_file_path = r'C:\Users\nicol\Documents\PoliTo\AdvancedML\project\SPair-71k.zip' 
extract_dir = r'C:\Users\nicol\Documents\PoliTo\AdvancedML\project\SPair-71k_extracted'

# Crea la directory di estrazione se non esiste
os.makedirs(extract_dir, exist_ok=True)

# Estrai il file ZIP solo se esiste
if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"File '{zip_file_path}' estratto con successo nella directory '{extract_dir}'")
    print(f"Contenuti della directory '{extract_dir}':\n{os.listdir(extract_dir)}")
else:
    print(f"File zip '{zip_file_path}' non trovato. Assicurati che il dataset sia estratto in '{extract_dir}'.")



File 'C:\Users\nicol\Documents\PoliTo\AdvancedML\project\SPair-71k.zip' estratto con successo nella directory 'C:\Users\nicol\Documents\PoliTo\AdvancedML\project\SPair-71k_extracted'
Contenuti della directory 'C:\Users\nicol\Documents\PoliTo\AdvancedML\project\SPair-71k_extracted':
['SPair-71k']


In [3]:
import os
import glob
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- 1. Define the Augmentation Pipeline ---
def get_transforms(mode='train', img_size=518):
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)

    if mode == 'train':
        return A.Compose([
            # Geometric Augmentations (Hard - Moves Keypoints)
            A.Resize(height=img_size, width=img_size), # Force DINOv2 size
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
            
            # Pixel Augmentations (Safe - Colors only)
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
            A.GaussianBlur(p=0.1),
            
            # Normalization & Conversion
            A.Normalize(mean=mean, std=std),
            ToTensorV2(), # Converts to (C, H, W)
        ], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))
    
    else:
        # Validation/Test: Only Resize & Normalize
        return A.Compose([
            A.Resize(height=img_size, width=img_size),
            A.Normalize(mean=mean, std=std),
            ToTensorV2(),
        ], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# --- 2. Simple Image Reader ---
def read_img(path):
    # Keep as HWC (Standard for Albumentations)
    # Do not transpose or convert to Tensor here yet
    img = np.array(Image.open(path).convert('RGB'))
    return img

class SPairDataset(Dataset):
    def __init__(self, pair_ann_path, layout_path, image_path, dataset_size, pck_alpha, datatype):
        self.datatype = datatype
        self.pck_alpha = pck_alpha
        self.ann_files = open(os.path.join(layout_path, dataset_size, datatype + '.txt'), "r").read().split('\n')
        self.ann_files = [x for x in self.ann_files if x] # Remove empty strings
        self.pair_ann_path = pair_ann_path
        self.image_path = image_path
        
        # Initialize the Transform Pipeline
        mode = 'train' if datatype == 'trn' else 'test'
        self.transform = get_transforms(mode=mode, img_size=518)

    def __len__(self):
        return len(self.ann_files)

    def __getitem__(self, idx):
        raw_line = self.ann_files[idx]
        ann_filename = raw_line.replace(':', '_')
        ann_file = ann_filename + '.json'
        json_path = os.path.join(self.pair_ann_path, self.datatype, ann_file)

        with open(json_path) as f:
            annotation = json.load(f)

        category = annotation['category']
        src_path = os.path.join(self.image_path, category, annotation['src_imname'])
        trg_path = os.path.join(self.image_path, category, annotation['trg_imname'])

        # 1. Load Images
        src_img_raw = read_img(src_path)
        trg_img_raw = read_img(trg_path)

        # 2. Get Keypoints
        src_kps = np.array(annotation['src_kps']).astype(np.float32)
        trg_kps = np.array(annotation['trg_kps']).astype(np.float32)

        # 3. Apply Augmentations
        src_aug = self.transform(image=src_img_raw, keypoints=src_kps)
        src_tensor = src_aug['image']
        src_kps_aug = np.array(src_aug['keypoints'])
        
        trg_aug = self.transform(image=trg_img_raw, keypoints=trg_kps)
        trg_tensor = trg_aug['image']
        trg_kps_aug = np.array(trg_aug['keypoints'])

        # ==========================================================
        # ⚠️ CRITICAL FIX: PADDING LOGIC (Prevents Stack Error)
        # ==========================================================
        # We enforce a fixed size of 40 points per image.
        MAX_KPS = 40 
        
        # Create empty containers filled with zeros (Shape: [40, 2])
        src_kps_padded = np.zeros((MAX_KPS, 2), dtype=np.float32)
        trg_kps_padded = np.zeros((MAX_KPS, 2), dtype=np.float32)
        
        # Get the actual number of points (limit to 40 just in case)
        n_src = min(len(src_kps_aug), MAX_KPS)
        n_trg = min(len(trg_kps_aug), MAX_KPS)
        
        # Copy the real points into the empty container
        if n_src > 0:
            src_kps_padded[:n_src] = src_kps_aug[:n_src]
        if n_trg > 0:
            trg_kps_padded[:n_trg] = trg_kps_aug[:n_trg]

        # Check which points are inside the image (Visibility)
        src_vis = self._check_visibility(src_kps_padded, 518, 518)
        trg_vis = self._check_visibility(trg_kps_padded, 518, 518)
        
        # Create the Valid Mask
        # A point is valid ONLY if:
        # 1. It existed in the original file (index < n_src)
        # 2. It is still inside the image boundaries (vis=1)
        valid_mask = np.zeros(MAX_KPS, dtype=np.float32)
        
        # We assume points are ordered pairs (1st src matches 1st trg)
        # So we only mark as valid if BOTH exist and are visible
        common_len = min(n_src, n_trg)
        valid_mask[:common_len] = src_vis[:common_len] * trg_vis[:common_len]
        # ==========================================================

        pck_threshold = 518 * self.pck_alpha

        sample = {
            'pair_id': annotation['pair_id'],
            'src_img': src_tensor,
            'trg_img': trg_tensor,
            
            # Now these are ALWAYS [40, 2], so PyTorch won't crash!
            'src_kps': torch.from_numpy(src_kps_padded).float(), 
            'trg_kps': torch.from_numpy(trg_kps_padded).float(), 
            'valid_mask': torch.from_numpy(valid_mask).float(), 
            
            'pck_threshold': pck_threshold,
            'category': category
        }

        return sample

    def _check_visibility(self, kps, h, w):
        """Returns a binary mask (N,) where 1=visible, 0=out of bounds"""
        # kps is shape (N, 2) -> (x, y)
        x = kps[:, 0]
        y = kps[:, 1]
        
        # Check boundaries
        vis_x = (x >= 0) & (x < w)
        vis_y = (y >= 0) & (y < h)
        return (vis_x & vis_y).astype(np.float32)

if __name__ == '__main__':
    # Update this path to your actual path
    base_dir = r"C:\Users\nicol\Documents\PoliTo\AdvancedML\project\SPair-71k_extracted\SPair-71k\SPair-71k"    
    
    pair_ann_path = os.path.join(base_dir, 'PairAnnotation')
    layout_path = os.path.join(base_dir, 'Layout')
    image_path = os.path.join(base_dir, 'JPEGImages')

    # Check paths
    if os.path.exists(base_dir):
        
        # --- 1. Load TRAINING Set ---
        print("Loading Training Set...")
        trn_dataset = SPairDataset(
            pair_ann_path, layout_path, image_path, 
            dataset_size='large', pck_alpha=0.05, 
            datatype='trn'  # <--- Loads from trn.txt
        )
        # SHUFFLE = TRUE for training (important for learning)
        trn_loader = DataLoader(trn_dataset, batch_size=4, shuffle=True)


        # --- 2. Load VALIDATION Set ---
        print("Loading Validation Set...")
        val_dataset = SPairDataset(
            pair_ann_path, layout_path, image_path, 
            dataset_size='large', pck_alpha=0.05, 
            datatype='val'  # <--- Loads from val.txt
        )
        # SHUFFLE = FALSE for validation (keep order stable)
        val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

        
        # --- 3. Test Loading ---
        print("Testing batches...")
        
        # Grab a training batch
        trn_batch = next(iter(trn_loader))
        print(f"Train Batch Images: {trn_batch['src_img'].shape}")
        
        # Grab a validation batch
        val_batch = next(iter(val_loader))
        print(f"Val Batch Images:   {val_batch['src_img'].shape}")
        
        print("Dataset setup complete. Ready for Training Loop.")
        
    else:
        print(f"Path not found: {base_dir}")

Loading Training Set...
Loading Validation Set...
Testing batches...
Train Batch Images: torch.Size([4, 3, 518, 518])
Val Batch Images:   torch.Size([4, 3, 518, 518])
Dataset setup complete. Ready for Training Loop.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

# --- 1. Define the Loss Function (InfoNCE) ---
def info_nce_loss(feat_A, feat_B, temperature=0.1):
    """
    Calculates the loss assuming the correct match for patch[i] in A 
    is patch[i] in B (The Diagonal).
    """
    # Normalize features (Crucial for Dot Product Similarity)
    feat_A = F.normalize(feat_A, dim=-1)
    feat_B = F.normalize(feat_B, dim=-1)
    
    # Similarity Matrix: [Batch, Tokens, Tokens]
    # We compare every patch in A against every patch in B
    sim_matrix = torch.bmm(feat_A, feat_B.transpose(1, 2)) / temperature
    
    # Labels: We assume Identity matching for the check (0->0, 1->1...)
    B, T, _ = sim_matrix.shape
    labels = torch.arange(T).to(feat_A.device).expand(B, T)
    
    # Calculate Cross Entropy
    loss = F.cross_entropy(sim_matrix.flatten(0, 1), labels.flatten())
    return loss

# --- 2. The Sanity Check ---
if __name__ == '__main__':
    # (Assuming trn_loader is defined from your previous code...)
    # from your_dataset_script import trn_loader 
    
    print("\n--- Step 1: InfoNCE Sanity Check ---")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 1. Initialize Model (Standard DINOv2 base)
    backbone = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14').to(device)
    backbone.eval() # Freeze for check
    
    # 2. Get One Batch
    try:
        batch = next(iter(trn_loader))
    except NameError:
        print("Error: 'trn_loader' is not defined. Run the Dataset setup code first!")
        exit()

    src_img = batch['src_img'].to(device)
    trg_img = batch['trg_img'].to(device)
    
    with torch.no_grad():
        # 3. Forward Pass
        dict_A = backbone.forward_features(src_img)
        dict_B = backbone.forward_features(trg_img)
        
        # Extract Patch Tokens: [Batch, 1369, 384]
        feat_A = dict_A['x_norm_patchtokens']
        feat_B = dict_B['x_norm_patchtokens']
        
        # 4. Calculate Loss
        loss = info_nce_loss(feat_A, feat_B)
        
    print(f"Calculated InfoNCE Loss: {loss.item():.4f}")
    
    # --- INTERPRETATION ---
    print("-" * 30)
    expected_loss = 7.22 # ln(1369)
    
    if 6.0 < loss.item() < 8.5:
        print(f"✅ PASSED. Loss is close to expected random guess ({expected_loss}).")
    elif loss.item() > 9.0:
        print(f"⚠️ HIGH. Loss is {loss.item():.2f}. This is normal if images are very different (not aligned).")
    else:
        print(f"❌ LOW. Loss is {loss.item():.2f}. Something is wrong (Masking? Identity leakage?).")


--- Step 1: InfoNCE Sanity Check ---


Using cache found in C:\Users\nicol/.cache\torch\hub\facebookresearch_dinov2_main


Calculated InfoNCE Loss: 6.5601
------------------------------
✅ PASSED. Loss is close to expected random guess (7.22).
